# Deep Convolutional Generative Adversarial Network (DCGAN)

## Ressources

In [ ]:
%matplotlib inline
import importlib
import utils; importlib.reload(utils)
from keras.layers import *
from keras.models import Sequential, Model
from keras.optimizers import Adam
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from utils import noise, limit_mem, train
import keras

In [ ]:
limit_mem()

## Data

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape

In [ ]:
X_train = X_train.reshape(60000, 28, 28, 1); X_train.shape

## Train

### Generator (G)

In [ ]:
x_len = 100

#### Model

In [ ]:
CNN_G = Sequential([
    Dense(512*7*7, input_dim=x_len),
    LeakyReLU(),
    BatchNormalization(),
    Reshape((7, 7, 512)),
    UpSampling2D(),
    Conv2D(64, 3, padding='same'),
    LeakyReLU(),
    BatchNormalization(),
    UpSampling2D(),
    Conv2D(32, 3, padding='same'),
    LeakyReLU(),
    BatchNormalization(),
    Conv2D(1, 1, padding='same', activation='sigmoid')
])

### Discriminator (D)

### Model

In [ ]:
CNN_D = Sequential([
    Conv2D(256, 5, strides=(2, 2), padding='same', input_shape=X_train.shape[1:]),
    LeakyReLU(),
    Conv2D(512, 5, strides=(2, 2), padding='same'),
    LeakyReLU(),
    Flatten(),
    Dense(256),
    LeakyReLU(),
    Dense(1, activation='sigmoid')
])

CNN_D.compile(Adam(lr=1e-3), 'binary_crossentropy')

### Model

We train the discriminator so it can at least tell real images from random noise

In [ ]:
sz = X_train.shape[0]//200
x1 = np.concatenate([np.random.permutation(X_train)[:sz], CNN_G.predict(noise(sz, x_len))])
x1.shape

In [ ]:
CNN_D.fit(x1, [0]*sz + [1]*sz, batch_size=128, epochs=1, verbose=2)

In [ ]:
CNN_m = Sequential([CNN_G, CNN_D])
CNN_m.compile(Adam(lr=1e-3), 'binary_crossentropy')

In [ ]:
CNN_m.layers[0].summary()

In [ ]:
K.set_value(CNN_D.optimizer.lr, 1e-3)
K.set_value(CNN_m.optimizer.lr, 1e-3)

In [ ]:
dl, gl = train(CNN_D, CNN_G, CNN_m, epochs=2500 ,x=X_train)